In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, TensorBoard
from keras.preprocessing import image
from tensorflow.keras.applications.resnet import ResNet101
from keras.layers import *
from keras.optimizers import *
from keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import random
# import splitfolders
import os

In [5]:
datadir = 'dataset'
categories = os.listdir(datadir)

In [6]:
len(categories)

30

In [7]:
data = []
for category in categories:
  path = os.path.join(datadir, category)
  labels = categories.index(category)
  for imgpath in os.listdir(path):
    imgs = image.load_img(os.path.join(path, imgpath),target_size=(224,224))
    data.append((imgs, labels))

In [8]:
len(data)

1835

In [9]:
random.shuffle(data)

In [10]:
X_img = []
y_lab = []
for img , label in data:
  X_img.append(img)
  y_lab.append(label)

In [11]:
X_list = [ image.img_to_array(img) for img in X_img]

In [12]:
X = np.asarray(X_list)

In [13]:
X.shape

(1835, 224, 224, 3)

In [14]:
X /= 255

In [15]:
y = np.asarray(y_lab)

In [16]:
y.shape

(1835,)

In [17]:
classes = 30
batch_size = 32
img_height = 224
img_width = 224

In [18]:
def build_model():

  base_model = ResNet101(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))

  xcp = base_model.output
  xcp = GlobalAveragePooling2D()(xcp)
  xcp = Dense(1024, activation='relu')(xcp)
  predictions = Dense(classes, activation = 'softmax')(xcp)

  model = Model(base_model.input, predictions)

  for layer in base_model.layers:
    layer.trainable = False

  return model

In [19]:
from sklearn.model_selection import KFold

In [20]:
kf = KFold(n_splits=5)

In [ ]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

count = 1
for train_index, test_index in kf.split(X):

  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  model = build_model()

  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  callbacks = [EarlyStopping(monitor='val_accuracy', patience=2,verbose=1),
              TensorBoard(log_dir=f'./logs{count}')
             ]

  history = model.fit(X_train ,y_train,epochs = 10,batch_size = 32,verbose = 2,callbacks= callbacks ,validation_data=(X_test,y_test))
  
  # results = model.evaluate(X_test, y_test)  #returns ['loss','accuracy']
  # print('fold = ',count, 'loss and accuracy',results)


  # VALIDATION_ACCURACY.append(results[1])
  # VALIDATION_LOSS.append(results[0])

  count += 1
  
 

Epoch 1/10
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
46/46 - 164s - loss: 3.1199 - accuracy: 0.1533 - val_loss: 2.6991 - val_accuracy: 0.2480
Epoch 2/10
46/46 - 161s - loss: 2.4884 - accuracy: 0.2868 - val_loss: 2.2988 - val_accuracy: 0.2507
Epoch 3/10
46/46 - 159s - loss: 2.1186 - accuracy: 0.3672 - val_loss: 2.0487 - val_accuracy: 0.3787
Epoch 4/10
46/46 - 159s - loss: 1.8477 - accuracy: 0.4898 - val_loss: 1.8202 - val_accuracy: 0.4659
Epoch 5/10
46/46 - 158s - loss: 1.6640 - accuracy: 0.5232 - val_loss: 1.5864 - val_accuracy: 0.5232
Epoch 6/10
46/46 - 189s - loss: 1.5027 - accuracy: 0.5749 - val_loss: 1.5684 - val_accuracy: 0.5504
Epoch 7/10
46/46 - 260s - loss: 1.3891 - accuracy: 0.5790 - val_loss: 1.5040 - val_accuracy: 0.5395
Epoch 8/10
46/46 - 257s - loss: 1.2814 - accuracy: 0.6328 - val_loss: 1.3834 - val_accuracy: 0.6104
Epoch 9/10
46/46 - 257s - loss: 1.2121 - accuracy: 0.6458 - val_loss: 1.2728 - val_accuracy: 0.6185
Epoch 10/10
46/46 - 262s - l